In [1]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
# model = SentenceTransformer("all-mpnet-base-v2")
model = SentenceTransformer("multi-qa-distilbert-cos-v1")


/home/alex/Projects/llm-24/.llm-24-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
user_question = "I just discovered the course. Can I still join it?"
v = model.encode(user_question)
len(v)

768

In [3]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [4]:
course = "machine-learning-zoomcamp"
documents = [doc for doc in documents if doc["course"] == course]

len(documents)

375

In [5]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [6]:
embeddings = []

for doc in documents:
    question = doc["question"]
    text = doc["text"]
    qa_text = f'{question} {text}'
    emb = model.encode(qa_text)
    embeddings.append(emb)


In [7]:
import numpy as np

X = np.array(embeddings)
X.shape

(375, 768)

In [8]:
scores = X.dot(v)

In [9]:
max(scores)

0.65065736

In [23]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, vector, num_results=10):
        scores = self.embeddings.dot(vector)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693',
  'question_text_vector': array([ 8.06286260e-02, -6.66388348e-02,  2.52730958e-02, -1.30044455e-02,
          7.58728385e-02, -5.94626442e-02, -2.18839068e-02,  2.90002185e-03,
          7.92940788e-04, -5.22237737e-03, -3.36517617e-02, -2.79135369e-02,
          5.81165925e-02,  3.97482552e-02,  5.44185415e-02, -3.82516421e-02,
          6.30574003e-02, -3.89698967e-02,  4.33617475e-04, -3.55845541e-02,
          1.75640546e-03

In [11]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [12]:
from tqdm.auto import tqdm

In [25]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


# hit_rate(relevance_total), mrr(relevance_total)

def evaluate(ground_truth, search_fn):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q["document"]
        v = model.encode(q["question"])
        results = search_fn(vector=v)
        # results = search_engine.search(v, num_results=5)
        relevance = [d["id"] == doc_id for d in results]
        relevance_total.append(relevance)
    return hit_rate(relevance_total)

# def vector_search(v):
#     return search_engine.search(v, num_results=5)

from functools import partial

vector_search = partial(search_engine.search, num_results=5)
evaluate(ground_truth, vector_search)

In [16]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch(['http://localhost:9200']) 

es_client.info()

ObjectApiResponse({'name': 'fd554bcd5381', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'rU3U-HY3RXaGT00knM9jvg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [17]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [18]:
for index in es_client.cat.indices(format='json'):
    print(index['index'])

course-questions


In [19]:
for doc, emb in zip(tqdm(documents), embeddings):
    doc["question_text_vector"] = emb
    es_client.index(index=index_name, document=doc)

100%|██████████| 375/375 [00:03<00:00, 97.43it/s] 


In [20]:
!curl -X GET "localhost:9200/_cat/indices?v"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


health status index            uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   course-questions t6f7RNcjSNSIsKN8cPj_pg   1   0        375            0       225b           225b


In [21]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs


elastic_search_knn(field="question_text_vector", vector=v, course=course)

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

In [26]:
vector_search = partial(elastic_search_knn, field="question_text_vector", course=course)
evaluate(ground_truth, vector_search)

100%|██████████| 1830/1830 [00:45<00:00, 40.54it/s]


0.9398907103825137